# PhoneLM

## `G2P` and `Encodec`

In [ ]:
!pip install g2p_en encodec

In [2]:
from g2p_en import G2p

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\win8t\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\cmudict.zip.


In [3]:
import torch
import random
import string
from functools import cache
import tqdm

@cache
def _get_model():
    return G2p()

@cache
def _get_graphs(path):
    with open(path, "r") as f:
        graphs = f.read()
    return graphs

def encode(graphs: str) -> list[str]:
    g2p = _get_model()
    phones = g2p(graphs)
    ignored = {" ", *string.punctuation}
    return ["_" if p in ignored else p for p in phones]

@torch.no_grad()
def write_phones(folder, suffix=".normalized.txt"):
    paths = list(folder.rglob(f"*{suffix}"))
    random.shuffle(paths)

    for path in tqdm(paths):
        phone_path = path.with_name(path.stem.split(".")[0] + ".phn.txt")
        if phone_path.exists():
            continue
        graphs = _get_graphs(path)
        phones = encode(graphs)
        with open(phone_path, "w") as f:
            f.write(" ".join(phones))